In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import log_loss
train = pd.read_csv('data/tap_fun_train.csv')
test = pd.read_csv('data/tap_fun_test.csv')
res0 = pd.DataFrame()
res0['user_id'] = test[test['pay_price']==0]['user_id']
res0['prediction_pay_price'] = 0


train = train[train['pay_price']>0]
test =test[test['pay_price']>0]




data = pd.concat([train,test],axis=0,ignore_index=True)

data['prediction_pay_price'].fillna(value=-1,inplace=True)


hour = []
register_date = []
register_day = []
for va in data['register_time'].values:
    hour.append(int(va.split(' ')[1].split(':')[0]))
    register_date.append(va.split(' ')[0])
    register_day.append(int(va.split(' ')[0].split('-')[2]))
data['register_hour'] = hour
data['register_date'] = register_date
data['register_day'] = register_day
#凌晨（0-6），上午（7-12），下午（13-18），晚上（19-23）
data['register_period'] = data['register_hour']//6

cols_value = []
for co in data.columns:
    if 'value' in co:
        cols_value.append(co)
n = len(cols_value)

for i in range(n//2):
    col_name = cols_value[2*i].replace('add','reserve')
    data[col_name] = data[cols_value[2*i]]-data[cols_value[2*i+1]]
    col_name = cols_value[2*i].replace('add_value','reduce_add_ratio')
    data[col_name] = data[cols_value[2*i+1]]/data[cols_value[2*i]]

data['sr_outpost_tier_level'] = data['sr_outpost_tier_2_level']+data['sr_outpost_tier_3_level']+data['sr_outpost_tier_4_level']

data['sr_outpost'] = data['sr_outpost_durability_level']*data['sr_outpost_tier_level']

data['sr_healing'] = data['sr_healing_space_level']*data['sr_healing_speed_level']



##统计特征
data['label'] = data['prediction_pay_price']/data['pay_price']
for co in data.columns:
    if 'level' in co or co in ['register_day','register_date','register_hour','register_period']:
        se = data[co].value_counts()
        data[co+'_cnt'] =  data[co].map(se)
        se = data.groupby(co)['pay_price'].mean()
        data[co+'_avg_pay'] = data[co].map(se)
        se = data.groupby(co)['avg_online_minutes'].mean()
        data[co+'_avg_online_minutes'] = data[co].map(se)
        se = data.groupby(co)['label'].mean()
        data[co+'_avg_label'] = data[co].map(se)
##week_day特征
register_date = data['register_date'].values.tolist()
se = pd.Series(pd.Series(register_date).drop_duplicates().sort_values().values)
se = pd.Series(se.index+5,index = se.values)%7
data['week_day'] = data['register_date'].map(se)
for co in ['register_time','register_date']:
    del data[co]
##pvp特征
data['pay_avg'] = data['pay_price']/data['pay_count']

data['pvp_win_battle_ratio'] = data['pvp_win_count']/data['pvp_battle_count']
data['pvp_lanch_battle_ratio'] = data['pvp_lanch_count']/data['pvp_battle_count']

data['pve_win_battle_ratio'] = data['pve_win_count']/data['pve_battle_count']
data['pve_lanch_battle_ratio'] = data['pve_lanch_count']/data['pve_battle_count']

data['pve_pvp_lanch_ratio'] = data['pve_lanch_count']/(data['pve_lanch_count']+data['pvp_lanch_count'])
data['pve_pvp_battle_ratio'] = data['pve_battle_count']/(data['pve_battle_count']+data['pvp_battle_count'])
data['pve_pvp_win_ratio'] = data['pve_win_count']/(data['pve_win_count']+data['pvp_win_count'])




for num in [10,20,30,50,70,100,200,500,1000,2000,5000,10000]:
    data['(pay_price)>'+str(num)] = (data['pay_price']>=num).astype(int)

type_feature = ['week_day','register_period','register_hour']
for co in type_feature:
    nums = data[co].drop_duplicates().values.tolist()
    for num in nums:
        data[co+'_'+str(num)] = (data[co]==num).astype(int)
    del data[co]
train_x = data[data['prediction_pay_price']!=-1]
test_x = data[data['prediction_pay_price']==-1]
print(train_x.shape)
print (test_x.shape)


label = ['label','prediction_pay_price']
k = 0

res = pd.DataFrame()
res['user_id'] = test_x['user_id'].values
del train_x['user_id']
del test_x['user_id']
del train_x[label[1-k]]
del test_x[label[1-k]]
train_y = train_x.pop(label[k])
del test_x[label[k]]

lst = train_y.sort_values().index.tolist()
train_index = []
test_index = []
se = pd.Series(lst)
se = pd.Series(se.index,index=se.values)
for i in range(5):
    test_index.append(se[se%5==i].index.tolist())
    train_index.append(se[se%5!=i].index.tolist())

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import log_loss

train_weight = train_x['pay_price']
test_weight = test_x['pay_price']


params_initial = {
        'max_depth':6, 
        'eta':0.01, 
    'booster':'gbtree',
  'min_child_weight':1,
    'subsample':0.8, 
    'colsample_bytree':0.8, 
    'reg_alpha':0,
    'reg_lambda':1, 
    'seed':2018,
    'eval_metric':'rmse',
    'objective':'reg:linear',
}
num_round=10000
print('##############################')
print('开始训练')
print(train_x.shape)
print (test_x.shape)
def five_flod_rmse(params,k,cols):
    rmse = []
    for i in range(5):
        if k==1:
            train_part = xgb.DMatrix(train_x[cols].loc[train_index[i]],weight=train_weight.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = xgb.DMatrix(train_x[cols].loc[test_index[i]],weight=train_weight.loc[test_index[i]],label=train_y.loc[test_index[i]])
        elif k==0:
            train_part = xgb.DMatrix(train_x[cols].loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = xgb.DMatrix(train_x[cols].loc[test_index[i]],label=train_y.loc[test_index[i]])
        bst = xgb.train(params,train_part,num_round,[(train_part,'train'),(evals,'evals')],verbose_eval=50,
                        maximize=False,early_stopping_rounds=20)
        rmse.append(bst.best_score)
    print(rmse)
    mean_rmse = np.array(rmse).mean()
    print(mean_rmse)
    return mean_rmse
col_all = train_x.columns.tolist()
rmse_initial = five_flod_rmse(params_initial,0,col_all)

(41439, 480)
(19549, 480)
##############################
开始训练
(41439, 477)
(19549, 477)
[0]	train-rmse:26.197	evals-rmse:22.9739
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:23.5123	evals-rmse:22.7951
[100]	train-rmse:21.4264	evals-rmse:22.7518
Stopping. Best iteration:
[88]	train-rmse:21.9136	evals-rmse:22.7485

[0]	train-rmse:25.9589	evals-rmse:23.9964
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:23.5614	evals-rmse:23.8628
[100]	train-rmse:21.4049	evals-rmse:23.8424
Stopping. Best iteration:
[102]	train-rmse:21.3069	evals-rmse:23.8409

[0]	train-rmse:24.7079	evals-rmse:28.8697
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:22.6951	evals-rmse:28

In [5]:
def get_best_params(params):
    col_all = train_x.columns.tolist()
    params_choose = {}
    params_choose['max_depth'] = list(range(3,10,1))
    params_choose['min_child_weight'] = list(range(10,110,10))
    params_choose['colsample_bytree'] = list(np.array(list(range(1,11)))*0.1)
    params_choose['subsample'] = list(np.array(list(range(1,11)))*0.1)
    params_choose['reg_alpha'] = list(range(0,11,2))
    params_choose['reg_lambda'] = list(range(0,11,2))
    params_choose['eta'] = list(np.array(list(range(1,11)))*0.01)
    for key in params_choose.keys():
        value = params_choose[key]
        score = pd.Series()
        for va in value:
            params[key] = va
            score = score.append(pd.Series(five_flod_rmse(params,0,col_all),index=[va]))
        score = score.sort_values()
        best_score = score.values[0]
        params[key] = score.index[0]
        pd.Series(params).to_csv('temp/temp_params_xgb_0.csv')
        print(key,best_score)
        print('\n')
    return params
import os
if os.path.exists('best/best_params_xgb_0.csv'):
    best_params = pd.read_csv('best/best_params_xgb_0.csv',header=None).set_index(0).to_dict()[1]
else:
    best_params = get_best_params(params_initial)
    pd.Series(best_params).to_csv('best/best_params_xgb_0.csv')

[0]	train-rmse:26.2414	evals-rmse:22.9744
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.0484	evals-rmse:22.8049
[100]	train-rmse:25.9369	evals-rmse:22.7378
[150]	train-rmse:25.8508	evals-rmse:22.7082
[200]	train-rmse:25.7805	evals-rmse:22.6941
[250]	train-rmse:25.7067	evals-rmse:22.6885
Stopping. Best iteration:
[247]	train-rmse:25.712	evals-rmse:22.6879

[0]	train-rmse:26.0116	evals-rmse:23.996
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.8222	evals-rmse:23.8327
[100]	train-rmse:25.7135	evals-rmse:23.7614
[150]	train-rmse:25.6293	evals-rmse:23.731
[200]	train-rmse:25.5673	evals-rmse:23.7098
[250]	train-rmse:25.5044	evals-rmse:23.7059
[300]	train-rmse:25.4518	evals-rmse:23.699
Stopping. Best iteration:
[303]	train-rmse:25.4487	evals-rmse:23.6985

[0]	tr

[100]	train-rmse:25.6401	evals-rmse:23.7519
[150]	train-rmse:25.5259	evals-rmse:23.724
[200]	train-rmse:25.4421	evals-rmse:23.713
Stopping. Best iteration:
[227]	train-rmse:25.3997	evals-rmse:23.7059

[0]	train-rmse:24.7427	evals-rmse:28.8701
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.5356	evals-rmse:28.7103
[100]	train-rmse:24.4163	evals-rmse:28.6392
[150]	train-rmse:24.3315	evals-rmse:28.5969
[200]	train-rmse:24.2684	evals-rmse:28.575
[250]	train-rmse:24.2131	evals-rmse:28.5592
[300]	train-rmse:24.1655	evals-rmse:28.5474
[350]	train-rmse:24.1212	evals-rmse:28.5393
Stopping. Best iteration:
[356]	train-rmse:24.1165	evals-rmse:28.5387

[0]	train-rmse:24.3056	evals-rmse:30.3183
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.0581	evals-rmse:30.1917
[100]

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.4835	evals-rmse:20.4167
[100]	train-rmse:26.328	evals-rmse:20.3376
[150]	train-rmse:26.2019	evals-rmse:20.311
Stopping. Best iteration:
[168]	train-rmse:26.1613	evals-rmse:20.3086

[22.692799, 23.704391, 28.546864, 30.140102, 20.30863]
25.0785572
[0]	train-rmse:26.2396	evals-rmse:22.9731
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.9982	evals-rmse:22.8009
[100]	train-rmse:25.8556	evals-rmse:22.7382
[150]	train-rmse:25.7462	evals-rmse:22.709
[200]	train-rmse:25.6562	evals-rmse:22.6985
[250]	train-rmse:25.5637	evals-rmse:22.696
Stopping. Best iteration:
[239]	train-rmse:25.5834	evals-rmse:22.6949

[0]	train-rmse:26.0099	evals-rmse:23.9961
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rms

Stopping. Best iteration:
[205]	train-rmse:25.6598	evals-rmse:23.7062

[0]	train-rmse:24.7425	evals-rmse:28.8696
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.5556	evals-rmse:28.6971
[100]	train-rmse:24.4752	evals-rmse:28.6374
[150]	train-rmse:24.422	evals-rmse:28.6058
[200]	train-rmse:24.3962	evals-rmse:28.6018
[250]	train-rmse:24.3719	evals-rmse:28.5827
Stopping. Best iteration:
[265]	train-rmse:24.3639	evals-rmse:28.5792

[0]	train-rmse:24.3046	evals-rmse:30.3169
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.1087	evals-rmse:30.1913
[100]	train-rmse:24.025	evals-rmse:30.1456
[150]	train-rmse:23.9735	evals-rmse:30.1295
Stopping. Best iteration:
[163]	train-rmse:23.9652	evals-rmse:30.1278

[0]	train-rmse:26.721	evals-rmse:20.6408
Multiple eval metrics ha

[50]	train-rmse:26.0056	evals-rmse:22.7985
[100]	train-rmse:25.8629	evals-rmse:22.7348
[150]	train-rmse:25.7593	evals-rmse:22.7133
[200]	train-rmse:25.6867	evals-rmse:22.7084
Stopping. Best iteration:
[223]	train-rmse:25.6503	evals-rmse:22.7048

[0]	train-rmse:26.0104	evals-rmse:23.9965
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.7699	evals-rmse:23.8209
[100]	train-rmse:25.6429	evals-rmse:23.7516
[150]	train-rmse:25.5515	evals-rmse:23.7291
[200]	train-rmse:25.4683	evals-rmse:23.7203
Stopping. Best iteration:
[192]	train-rmse:25.4769	evals-rmse:23.7191

[0]	train-rmse:24.7429	evals-rmse:28.8706
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.5275	evals-rmse:28.7046
[100]	train-rmse:24.4168	evals-rmse:28.6417
[150]	train-rmse:24.3336	evals-rmse:28.6069
[20

...

[150]	train-rmse:23.7279	evals-rmse:30.137
Stopping. Best iteration:
[168]	train-rmse:23.6945	evals-rmse:30.1356

[0]	train-rmse:26.7212	evals-rmse:20.6404
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.4872	evals-rmse:20.4195
[100]	train-rmse:26.3332	evals-rmse:20.3392
[150]	train-rmse:26.2122	evals-rmse:20.3127
[200]	train-rmse:26.1066	evals-rmse:20.3051
Stopping. Best iteration:
[218]	train-rmse:26.0755	evals-rmse:20.3008

[22.692705, 23.698578, 28.539843, 30.135605, 20.300819]
25.073510000000002
[0]	train-rmse:26.2405	evals-rmse:22.9724
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.0053	evals-rmse:22.7988
[100]	train-rmse:25.8582	evals-rmse:22.7324
[150]	train-rmse:25.7522	evals-rmse:22.7039
[200]	train-rmse:25.6572	evals-rmse:22.6931
Stopping. Best i

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.7729	evals-rmse:23.8146
[100]	train-rmse:25.6243	evals-rmse:23.7445
[150]	train-rmse:25.5124	evals-rmse:23.7179
[200]	train-rmse:25.4347	evals-rmse:23.704
[250]	train-rmse:25.3598	evals-rmse:23.7001
Stopping. Best iteration:
[247]	train-rmse:25.3684	evals-rmse:23.6992

[0]	train-rmse:24.742	evals-rmse:28.8696
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.5284	evals-rmse:28.7083
[100]	train-rmse:24.4135	evals-rmse:28.6381
[150]	train-rmse:24.3299	evals-rmse:28.6019
[200]	train-rmse:24.2685	evals-rmse:28.5846
[250]	train-rmse:24.2108	evals-rmse:28.576
[300]	train-rmse:24.1641	evals-rmse:28.5639
[350]	train-rmse:24.1171	evals-rmse:28.5529
Stopping. Best iteration:
[350]	train-rmse:24.1171	evals-rmse:28.5529

[0]	train-rmse:24.3045	evals-rmse:30.3181
Multiple eval metrics have been passed: 'evals-r

Stopping. Best iteration:
[168]	train-rmse:23.7143	evals-rmse:30.1339

[0]	train-rmse:26.721	evals-rmse:20.6402
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.4915	evals-rmse:20.4238
[100]	train-rmse:26.3423	evals-rmse:20.3406
[150]	train-rmse:26.2226	evals-rmse:20.3119
[200]	train-rmse:26.1228	evals-rmse:20.3054
Stopping. Best iteration:
[217]	train-rmse:26.0899	evals-rmse:20.3016

[22.690676, 23.70355, 28.548229, 30.133942, 20.301573]
25.075594000000002
reg_lambda 25.074117199999996


[0]	train-rmse:26.2405	evals-rmse:22.9724
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:26.0046	evals-rmse:22.7986
[100]	train-rmse:25.8585	evals-rmse:22.7333
[150]	train-rmse:25.7509	evals-rmse:22.7053
[200]	train-rmse:25.6543	evals-rmse:22.6951
Stopping. Best iteration:
[19

[0]	train-rmse:26.2162	evals-rmse:22.946
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.612	evals-rmse:22.7179
Stopping. Best iteration:
[46]	train-rmse:25.6404	evals-rmse:22.7171

[0]	train-rmse:25.9848	evals-rmse:23.9733
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:25.3699	evals-rmse:23.7213
Stopping. Best iteration:
[38]	train-rmse:25.4548	evals-rmse:23.7148

[0]	train-rmse:24.7184	evals-rmse:28.8509
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
[50]	train-rmse:24.2253	evals-rmse:28.5738
Stopping. Best iteration:
[60]	train-rmse:24.171	evals-rmse:28.5519

[0]	train-rmse:24.2778	evals-rmse:30.3018
Multiple eval metrics have been passed: 'evals-rmse' will be 

[0]	train-rmse:26.6728	evals-rmse:20.589
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
Stopping. Best iteration:
[21]	train-rmse:26.0659	evals-rmse:20.2824

[22.697937, 23.699638, 28.518486, 30.152092, 20.282415]
25.0701136
eta 25.0663516




In [13]:
def get_predict_result(params,k,cols):
    df = pd.DataFrame()
    num = 0
    rmse = []
    test= xgb.DMatrix(test_x[cols],weight=test_weight)
    for i in range(5):
        if k==1:
            train_part = xgb.DMatrix(train_x[cols].loc[train_index[i]],weight=train_weight.loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = xgb.DMatrix(train_x[cols].loc[test_index[i]],weight=train_weight.loc[test_index[i]],label=train_y.loc[test_index[i]])
        elif k==0:
            train_part = xgb.DMatrix(train_x[cols].loc[train_index[i]],label=train_y.loc[train_index[i]])
            evals = xgb.DMatrix(train_x[cols].loc[test_index[i]],label=train_y.loc[test_index[i]])
        bst = xgb.train(params,train_part,num_round,[(train_part,'train'),(evals,'evals')],verbose_eval=50,
                        maximize=False,early_stopping_rounds=20)
        num+=1
        test_ypre = bst.predict(test)
        se = pd.Series(bst.get_fscore()).sort_values(ascending=False)
        print(se.head())
        df['pre_'+str(num)]=test_ypre
        rmse.append(bst.best_score)
    print(rmse)
    print(np.array(rmse).mean())
    return df,rmse
col_all = train_x.columns.tolist()
result,rmse = get_predict_result(best_params,0,col_all)
se = result.mean(1)
result['prediction_pay_price'] = se.values*test_x['pay_price'].values
result['user_id'] = res['user_id']
sub1 = result[['user_id','prediction_pay_price']]
sub1 = pd.concat([sub1,res0],axis=0,ignore_index=True)
score = round(np.array(rmse).mean(),3)
filename = 'result/positive_xgb_0_'+str(score)+'.csv'
sub1.to_csv(filename,index=False)
print(filename)
print('Done')

[0]	train-rmse:26.193	evals-rmse:22.9208
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
Stopping. Best iteration:
[20]	train-rmse:25.729	evals-rmse:22.6907

register_date_avg_label          19
avg_online_minutes               16
pay_price                        11
pve_win_count                    11
bd_hero_gacha_level_avg_label     9
dtype: int64
[0]	train-rmse:25.9604	evals-rmse:23.9522
Multiple eval metrics have been passed: 'evals-rmse' will be used for early stopping.

Will train until evals-rmse hasn't improved in 20 rounds.
Stopping. Best iteration:
[25]	train-rmse:25.3981	evals-rmse:23.708

register_date_avg_label            15
avg_online_minutes                 14
register_hour_avg_label            10
pve_pvp_lanch_ratio                 8
wound_infantry_reduce_add_ratio     7
dtype: int64
[0]	train-rmse:24.6959	evals-rmse:28.8329
Multiple eval metrics have been passed: 'evals-rmse